In [1]:
import gym
import time
from gym.envs.registration import register
import argparse
from ray.rllib.algorithms.ppo import PPOConfig
from ray.rllib.connectors.env_to_module import FlattenObservations
from marllib import marl

2025-05-04 20:47:26,265	INFO util.py:154 -- Outdated packages:
  ipywidgets==7.7.2 found, needs ipywidgets>=8
Run `pip install -U ipywidgets`, then restart the notebook server for rich notebook output.
2025-05-04 20:47:31,016	INFO util.py:154 -- Outdated packages:
  ipywidgets==7.7.2 found, needs ipywidgets>=8
Run `pip install -U ipywidgets`, then restart the notebook server for rich notebook output.


ModuleNotFoundError: No module named 'marllib'

In [2]:
register(
            id='multigrid-collect-v0',
            entry_point='gym_multigrid.envs:CollectGame4HEnv10x10N2',
        )
env = gym.make('multigrid-collect-v0')

In [7]:
_ = env.reset()
nb_agents = len(env.agents)

for _ in range(1):
    env.render(highlight=True)
    time.sleep(0.1)
    
    ac = [env.action_space.sample() for _ in range(nb_agents)]
    #ac = [0] * nb_agents
    
    obs, _, done, _ = env.step(ac)
    print("Num Agents = ", len(obs))
    print("Obs shape = ", obs[0].T.shape)
    
    if done:
        break

Num Agents =  3
Obs shape =  (6, 7, 7)
